In [2]:
# ติดตั้ง yfinance หากยังไม่มี
# !pip install yfinance 

import yfinance as yf
import pandas as pd
import numpy as np
import warnings

# ปิด Warning เพื่อความสะอาดของ Output
warnings.filterwarnings('ignore')

In [3]:
# รายชื่อหุ้น SET50
SET50_TICKERS = [
    "ADVANC.BK", "AOT.BK", "AWC.BK", "BANPU.BK", "BBL.BK", "BDMS.BK", "BEM.BK", "BGRIM.BK", "BH.BK", "BJC.BK",
    "BPP.BK", "CPALL.BK", "CPF.BK", "CPN.BK", "CRC.BK", "DELTA.BK", "EGCO.BK", "ESSO.BK", "GULF.BK", "HMPRO.BK",
    "IRPC.BK", "KBANK.BK", "KTB.BK", "KTC.BK", "LH.BK", "MINT.BK", "MTC.BK", "OR.BK", "OSP.BK",
    "PTT.BK", "PTTEP.BK", "PTTGC.BK", "RATCH.BK", "SAWAD.BK", "SCB.BK", "SCC.BK", "SCGP.BK", "TISCO.BK", "TLI.BK",
    "TOP.BK", "TTB.BK", "TU.BK", "VGI.BK", "WHA.BK", "GLOBAL.BK", "BAM.BK", "CPAXT.BK", "GPSC.BK", "BLA.BK"
]

In [4]:
def calculate_rsi(series, period=14):
    """
    คำนวณ RSI (Relative Strength Index)
    """
    delta = series.diff()
    gain = (delta.where(delta > 0, 0))
    loss = (-delta.where(delta < 0, 0))
    
    # ใช้ Exponential Moving Average (Wilder's Smoothing)
    avg_gain = gain.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

def calculate_macd(series, fast=12, slow=26, signal=9):
    """
    คำนวณ MACD (Moving Average Convergence Divergence)
    Return: MACD Line, Signal Line, Histogram
    """
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()
    
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    histogram = macd_line - signal_line
    
    return macd_line, signal_line, histogram

In [5]:
def get_technical_history(symbol: str, period: str = "1y"):
    """
    ดึงข้อมูลราคา + MACD + RSI แบบรายวัน (Time Series)
    """
    try:
        clean_symbol = symbol.upper().replace('.BK', '')
        ticker = f"{clean_symbol}.BK"
        
        # ดึงข้อมูลจาก yfinance
        stock = yf.Ticker(ticker)
        df = stock.history(period=period)
        
        if df.empty:
            return {"status": "error", "message": f"No data found for {symbol}"}
            
        # คำนวณ Indicators
        df['RSI'] = calculate_rsi(df['Close'])
        df['MACD'], df['Signal'], df['Hist'] = calculate_macd(df['Close'])
        
        # ลบค่า NaN ช่วงแรกที่คำนวณไม่ได้
        df = df.dropna()
        
        history_data = []
        for date, row in df.iterrows():
            # แปลงสัญญาณ MACD Histogram
            signal_status = "Neutral"
            if row['Hist'] > 0: signal_status = "Bullish"
            elif row['Hist'] < 0: signal_status = "Bearish"

            history_data.append({
                "Date": date.strftime('%Y-%m-%d'),
                "Close": round(row['Close'], 2),
                "RSI": round(row['RSI'], 2),
                "MACD": round(row['MACD'], 4),
                "Signal": round(row['Signal'], 4),
                "Hist": round(row['Hist'], 4),
                "Momentum": signal_status
            })
            
        return {
            "status": "success",
            "symbol": clean_symbol,
            "period": period,
            "count": len(history_data),
            "data": history_data
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}

In [6]:
def get_technical_snapshot(tickers: list = None):
    """
    ดึงค่า RSI/MACD ล่าสุด ของหุ้นหลายตัว (Batch Processing)
    """
    target_tickers = tickers if tickers else SET50_TICKERS
    results = []
    
    print(f"🔄 Processing {len(target_tickers)} stocks...")
    
    for symbol in target_tickers:
        try:
            clean_symbol = symbol.replace('.BK', '')
            ticker = f"{clean_symbol}.BK"
            
            stock = yf.Ticker(ticker)
            df = stock.history(period="6mo")
            
            if df.empty: continue
            
            # คำนวณ
            df['RSI'] = calculate_rsi(df['Close'])
            df['MACD'], df['Signal'], df['Hist'] = calculate_macd(df['Close'])
            
            # เอาค่าล่าสุด (Last) และก่อนหน้า (Previous) เพื่อดูจุด Cross
            last = df.iloc[-1]
            prev = df.iloc[-2]
            
            # วิเคราะห์สัญญาณ MACD Cross
            macd_signal = "Neutral"
            if last['MACD'] > last['Signal'] and prev['MACD'] <= prev['Signal']:
                macd_signal = "Golden Cross (Buy)"
            elif last['MACD'] < last['Signal'] and prev['MACD'] >= prev['Signal']:
                macd_signal = "Dead Cross (Sell)"
            
            # วิเคราะห์ RSI Status
            rsi_status = "Neutral"
            if last['RSI'] >= 70: rsi_status = "Overbought"
            elif last['RSI'] <= 30: rsi_status = "Oversold"
            
            results.append({
                "Stock": clean_symbol,
                "Date": last.name.strftime('%Y-%m-%d'),
                "Close": round(last['Close'], 2),
                "RSI": round(last['RSI'], 2),
                "RSI_Status": rsi_status,
                "MACD_Signal": macd_signal,
                "MACD_Hist": round(last['Hist'], 4)
            })
            
        except Exception as e:
            print(f"Error analyzing {symbol}: {e}")
            continue
            
    return {
        "status": "success",
        "count": len(results),
        "data": results
    }

In [7]:
# ทดสอบดึงข้อมูล KBANK
history_res = get_technical_history("KBANK", period="1y")

if history_res['status'] == 'success':
    df_history = pd.DataFrame(history_res['data'])
    print(f"📈 History Data for: {history_res['symbol']}")
    display(df_history.tail(5)) # แสดง 5 วันล่าสุด
else:
    print("Error:", history_res['message'])

📈 History Data for: KBANK


,Date,Close,RSI,MACD,Signal,Hist,Momentum
225,2025-12-02,190.0,67.76,3.0484,3.1501,-0.1017,Bearish
226,2025-12-03,191.0,69.50,3.1401,3.1481,-0.0081,Bearish
227,2025-12-04,191.0,69.50,3.1761,3.1537,0.0224,Bullish
228,2025-12-08,191.5,70.42,3.2080,3.1646,0.0434,Bullish
229,2025-12-09,191.5,70.42,3.1964,3.1709,0.0255,Bullish


In [8]:
# ทดสอบสแกนทั้งตลาด (ใช้เวลาสักครู่)
snapshot_res = get_technical_snapshot()

if snapshot_res['status'] == 'success':
    df_snapshot = pd.DataFrame(snapshot_res['data'])
    
    print(f"\n✅ Scan Complete! Total Stocks: {snapshot_res['count']}")
    
    # 1. แสดงหุ้นที่เกิดสัญญาณ Golden Cross (ซื้อ)
    golden_cross = df_snapshot[df_snapshot['MACD_Signal'] == 'Golden Cross (Buy)']
    print("\n🟢 Golden Cross Stocks:")
    display(golden_cross)

    # 2. แสดงหุ้นที่ Oversold (RSI <= 30)
    oversold = df_snapshot[df_snapshot['RSI_Status'] == 'Oversold']
    print("\n🔵 Oversold Stocks (RSI <= 30):")
    display(oversold)

    # 3. แสดงตารางรวมทั้งหมด
    print("\n📋 Full Table (First 5 rows):")
    display(df_snapshot.head())

🔄 Processing 49 stocks...


$ESSO.BK: possibly delisted; no price data found  (period=6mo) (Yahoo error = "No data found, symbol may be delisted")



✅ Scan Complete! Total Stocks: 48

🟢 Golden Cross Stocks:


,Stock,Date,Close,RSI,RSI_Status,MACD_Signal,MACD_Hist
17,GULF,2025-12-09,40.75,42.27,Neutral,Golden Cross (Buy),0.0445
46,GPSC,2025-12-09,34.75,38.38,Neutral,Golden Cross (Buy),0.0042



🔵 Oversold Stocks (RSI <= 30):


,Stock,Date,Close,RSI,RSI_Status,MACD_Signal,MACD_Hist
9,BJC,2025-12-09,15.1,29.63,Oversold,Neutral,0.0253
34,SCC,2025-12-09,178.0,29.75,Oversold,Neutral,0.2334
45,CPAXT,2025-12-09,15.6,22.23,Oversold,Neutral,0.0568



📋 Full Table (First 5 rows):


,Stock,Date,Close,RSI,RSI_Status,MACD_Signal,MACD_Hist
0,ADVANC,2025-12-09,310.0,51.65,Neutral,Neutral,-0.9985
1,AOT,2025-12-09,54.0,83.90,Overbought,Neutral,1.2364
2,AWC,2025-12-09,2.0,44.80,Neutral,Neutral,0.0132
3,BANPU,2025-12-09,5.0,66.02,Neutral,Neutral,0.0680
4,BBL,2025-12-09,163.5,61.99,Neutral,Neutral,0.5425
